In [1]:
import torch
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader

from pitch_tracker.utils import dataset, files
from pitch_tracker.utils.constants import (F_MIN, HOP_LENGTH, N_CLASS, N_FFT,
                                           N_MELS, PICKING_FRAME_SIZE,
                                           PICKING_FRAME_STEP,
                                           PICKING_FRAME_TIME, SAMPLE_RATE,
                                           STEP_FRAME, STEP_TIME, WIN_LENGTH)
from pitch_tracker.utils.dataset import AudioDataset
from pitch_tracker.ml.model.net import NeuralNetwork
from pitch_tracker.ml. import train_model, test_model

e:\Programs\miniconda3\envs\mpt_2022\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E:\Spaceship420\MY GIT\mono_pitch_tracker\medleydb\medleydb\__init__.py:69: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  INST_TAXONOMY = yaml.load(fhandle)
E:\Spaceship420\MY GIT\mono_pitch_tracker\medleydb\medleydb\__init__.py:77: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  MIXING_COEFFICIENTS = yaml.load(fhandle)

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


In [2]:
device = "cuda" if torch.cuda.is_available() \
    else "mps" if torch.backends.mps.is_available() \
    else "cpu"

# device = "cpu"

print(f"Using {device} device")

Using cuda device


In [3]:
DATASET_DIR = '../content/pickled_database/'

dataset_paths = list(files.list_folder_paths_in_dir(DATASET_DIR))
train_set, validation_set = train_test_split(dataset_paths, test_size=0.40, random_state=1, shuffle=True)
validation_set, test_set = train_test_split(validation_set, test_size=0.50, random_state=1, shuffle=True)
print(f'train_song_set: {len(train_set)}')
print(f'validation_song_set: {len(validation_set)}')
print(f'test_song_set: {len(test_set)}')

train_song_set: 64
validation_song_set: 22
test_song_set: 22


In [4]:
train_dataset = AudioDataset(train_set)
validation_dataset = AudioDataset(validation_set)
test_dataset = AudioDataset(test_set)

# affect GPU dedicated memory
batch_size = 4

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [5]:
model = NeuralNetwork().to(device)
# loss_fn = nn.BCELoss()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# print(model)

e:\Programs\miniconda3\envs\mpt_2022\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [6]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_model(model, validation_dataloader,loss_fn, optimizer, device)
    test_model(model, test_dataloader, loss_fn, device)
print("Done!")

Epoch 1
-------------------------------
[    1/  648]  Batch Accuracy: 6.2%, current loss: 13.060298
[   50/  648]  Batch Accuracy: 0.0%, current loss: 641.974011
[   99/  648]  Batch Accuracy: 0.5%, current loss: 1269.609948
[  148/  648]  Batch Accuracy: 0.0%, current loss: 1896.429015
[  197/  648]  Batch Accuracy: 0.0%, current loss: 2522.489054
[  246/  648]  Batch Accuracy: 0.0%, current loss: 3148.550686
[  295/  648]  Batch Accuracy: 1.4%, current loss: 3774.858865
[  344/  648]  Batch Accuracy: 1.4%, current loss: 4400.766545
[  393/  648]  Batch Accuracy: 1.0%, current loss: 5026.630705
[  442/  648]  Batch Accuracy: 0.0%, current loss: 5653.060153
[  491/  648]  Batch Accuracy: 0.0%, current loss: 6279.430711
[  540/  648]  Batch Accuracy: 1.0%, current loss: 6904.975354
[  589/  648]  Batch Accuracy: 0.0%, current loss: 7530.056245
[  638/  648]  Batch Accuracy: 0.0%, current loss: 8155.244347
[  648/  648]  Total Accuracy: 2.9%, Avg loss: 12.781859
Test Error: 
 Accuracy: 

In [64]:
test_model(model, validation_dataloader, loss_fn, device)


Test Error: 
 Accuracy: 3.6%, Avg loss: 0.300722 



In [23]:
X, (y1,y2,y3) = next(iter(train_dataloader))
X = X.to(device)
y3 = y3.to(device)
y_pred = model(X)

In [24]:
y_pred

tensor([[[-0.2972,  0.3336, -0.1609,  ..., -0.0524, -0.0204, -0.8517],
         [-0.1902,  0.0582,  0.1809,  ...,  0.0170,  0.2186, -0.5805],
         [-0.3692, -0.0507,  0.0995,  ..., -0.0526,  0.0991, -0.6397],
         ...,
         [-0.5017, -0.3042, -0.5330,  ..., -0.0360, -0.4683,  0.2623],
         [-0.4445,  0.0660, -0.5317,  ..., -0.0100, -0.4262,  0.6025],
         [-0.2346, -0.5030, -0.2007,  ..., -0.0692,  0.0729, -0.1966]]],
       grad_fn=<ViewBackward0>)

In [25]:
loss = loss_fn(y_pred, y3)

In [27]:
loss.item()

13.041638374328613

In [20]:
y_pred.size()
y3.size()

torch.Size([4, 210, 88])

In [47]:
y_pred.numel()

73920

In [51]:
y_pred.argmax(2).flatten() == y3.argmax(2).flatten()

torch.Size([840])

In [62]:
(y_pred.argmax(2) == y3.argmax(2)).type(torch.float).sum().item()

11.0

In [55]:
pos_neg_matrix = (y_pred.argmax(2) == y3.argmax(2)).flatten()
n_size = pos_neg_matrix.numel()
n_correct = torch.nonzero(pos_neg_matrix).numel()

In [8]:
from datetime import datetime


date_time = datetime.now().strftime('%Y%m%d-%H%M%S')
date_time

'20230310-171903'